BOOK RECOMMENDATION SYSTEM

SPRINT-2

In [22]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
from math import sqrt
from sklearn.metrics import mean_squared_error

In [12]:
book_rating=ratings.dropna(axis=0,subset=["Book-Title"])
book_rating_count=book_rating.groupby(["Book-Title"])["Book-Rating"].count().reset_index().rename(columns={'Book-Rating':'total_rating_count'})
book_rating_count.head()

,Book-Title,total_rating_count
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [13]:
ratings=ratings.merge(book_rating_count,on="Book-Title",how="right")

In [14]:
ratings.total_rating_count.describe()

count    1.031136e+06
mean     6.978162e+01
std      1.753381e+02
min      1.000000e+00
25%      3.000000e+00
50%      1.300000e+01
75%      6.100000e+01
max      2.502000e+03
Name: total_rating_count, dtype: float64

In [15]:
ratings.head()

,User-ID,ISBN,Book-Rating,Book-Title,total_rating_count
0,18995,0590567330,0,A Light in the Storm: The Civil War Diary of ...,4
1,35859,0590567330,0,A Light in the Storm: The Civil War Diary of ...,4
2,55927,0590567330,0,A Light in the Storm: The Civil War Diary of ...,4
3,96448,0590567330,9,A Light in the Storm: The Civil War Diary of ...,4
4,172742,0964147726,0,Always Have Popsicles,1


In [23]:
user_features_df=rating_popular_books_df.pivot_table(index="User-ID",columns="Book-Title",values="Book-Rating").fillna(0.0)
book_features_df=rating_popular_books_df.pivot_table(index="Book-Title",columns="User-ID",values="Book-Rating").fillna(0.0)

In [24]:
user_features_matrix = csr_matrix(user_features_df)
book_features_matrix = csr_matrix(book_features_df)

In [25]:
model_knn1 = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors = 20, n_jobs=-1)
model_knn1.fit(user_features_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [26]:
user_features_df.shape

(38132, 914)

In [27]:
user_features_df.head(10)

Book-Title,1984,1st to Die: A Novel,24 Hours,2nd Chance,4 Blondes,84 Charing Cross Road,A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",...,Without Remorse,"Wizard and Glass (The Dark Tower, Book 4)",Women Who Run with the Wolves,"Word Freak: Heartbreak, Triumph, Genius, and Obsession in the World of Competitive Scrabble Players",Wuthering Heights,Year of Wonders,You Belong To Me,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,Zoya,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
query_index=np.random.choice(user_features_df.shape[0])
print(query_index)

20250


In [29]:
distances, indices = model_knn1.kneighbors(user_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
user_ids = []
for index in range(0, len(distances.flatten())):
    user_ids.append(user_features_df.index[indices.flatten()[index]])
    if index == 0: # the movie chosen
        print(f"Users similar with user having user_id:  {user_features_df.index[query_index]}")
        print("---------------------------------------------------------")
    else:
        print(f"{index}: {user_features_df.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

In [ ]:
print(user_ids)

In [ ]:
rating_popular_books_df.info()

In [ ]:
# select books that were highly ranked by the most similar users.
# look only for books highly rated by the similar users, not the current user
candidate_user_ids = user_ids[1:]
sel_ratings = rating_popular_books_df.loc[rating_popular_books_df['User-ID'].isin(candidate_user_ids)]

# sort by best ratings and total rating count
sel_ratings = sel_ratings.sort_values(by=["Book-Rating", "total_rating_count"], ascending=False)

# eliminate from the selection books that were ranked already by the current user
books_rated_by_targeted_user = list(rating_popular_books_df.loc[rating_popular_books_df['User-ID'] == user_ids[0]]["ISBN"].values)
sel_ratings = sel_ratings.loc[~sel_ratings['ISBN'].isin(books_rated_by_targeted_user)]

# aggregate and count total ratings and total total_rating_count
agg_sel_ratings = sel_ratings.groupby(["Book-Title", "Book-Rating"])["total_rating_count"].max().reset_index()
agg_sel_ratings.columns = ["Book-Title", "Book-Rating", "total_ratings"]
agg_sel_ratings = agg_sel_ratings.sort_values(by=["Book-Rating", "total_ratings"], ascending=False)

# only select top n (default top 10 here)
rec_list = agg_sel_ratings["Book-Title"].head(10).values

print(f"\nBooks recommended to user: {indices[0][0]}\n---------------------------------")
for i, rec in enumerate(rec_list):
    print(f"{i+1}: {rec}")

In [ ]:
def recommend_user_based(user_id=10):
    n_users = 5
    rec_top_n = 10
    distances, indices = model_knn1.kneighbors(user_features_df.loc[user_features_df.index == user_id].values.reshape(1, -1), n_neighbors=n_users + 1)
    user_ids = []
    for index in range(0, len(distances.flatten())):
        user_ids.append(user_features_df.index[indices.flatten()[index]])
        if index == 0:  # the user chosen
            print(f"Users similar with user having user_id: {user_id}")
            print("---------------------------------------------------------")
        else:
            print(f"{index}: {user_features_df.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

    # select books that were highly ranked by the most similar users.

    # look only for books highly rated by the similar users, not the current user
    candidate_user_ids = user_ids[1:]
    sel_ratings = rating_popular_books_df.loc[rating_popular_books_df['User-ID'].isin(candidate_user_ids)]
    # sort by best ratings and total rating count
    sel_ratings = sel_ratings.sort_values(by=["Book-Rating", "total_rating_count"], ascending=False)
    # eliminate from the selection books that were ranked already by the current user
    books_rated_by_targeted_user = list(rating_popular_books_df.loc[rating_popular_books_df['User-ID'] == user_ids[0]]["ISBN"].values)
    sel_ratings = sel_ratings.loc[~sel_ratings['ISBN'].isin(books_rated_by_targeted_user)]
    # aggregate and count total ratings and total total_rating_count
    agg_sel_ratings = sel_ratings.groupby(["Book-Title", "Book-Rating"])["total_rating_count"].max().reset_index()
    agg_sel_ratings.columns = ["Book-Title", "Book-Rating", "total_ratings"]
    agg_sel_ratings = agg_sel_ratings.sort_values(by=["Book-Rating", "total_ratings"], ascending=False)
    # only select top n (default top 10 here)
    rec_list = agg_sel_ratings["Book-Title"].head(10).values
    print(f"\nBooks recommended to user_id: {user_ids[0]}\n---------------------------------")
    for i, rec in enumerate(rec_list):
        print(f"{i + 1}: {rec}")

In [ ]:
model_knn2 = NearestNeighbors(metric = "cosine", algorithm = "brute", n_neighbors = 20, n_jobs=-1)
model_knn2.fit(book_features_matrix)

In [ ]:
query_index1 = np.random.choice(book_features_df.shape[0])
print(query_index1)

In [ ]:
distances, indices = model_knn2.kneighbors(book_features_df.iloc[query_index1,:].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
for index in range(0, len(distances.flatten())):
    if index == 0: # the movie chosen
        print(f"Recommendation for {book_features_df.index[query_index1]} (dist: {distances.flatten()[index]})")
        print("---------------------------------------------------------")
    else:
        print(f"{index}: {book_features_df.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

In [ ]:
def recommenditembased(query_index=10):
    top_n=10
    distances, indices = model_knn2.kneighbors(book_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = top_n+1)
    for index in range(0, len(distances.flatten())):
        if index == 0: # the movie chosen
            print(f"Recommendation for {book_features_df.index[query_index]} (dist: {distances.flatten()[index]})")
            print("---------------------------------------------------------")
        else:
            print(f"{index}: {book_features_df.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

In [ ]:
def recommend_item_based(book_title, top_n=10):
    # Check if the book_title exists in the dataset
    if book_title not in book_features_df.index:
        print(f"The Book '{book_title}' was not found in the dataset.")
        return

    # Find the query_index for the given movie_title
    query_index = book_features_df.index.get_loc(book_title)

    distances, indices = model_knn2.kneighbors(book_features_df.iloc[query_index, :].values.reshape(1, -1), n_neighbors=top_n + 1)

    for index in range(0, len(distances.flatten())):
        if index == 0:  # the movie chosen
            print(f"Recommendation for {book_features_df.index[query_index]} (dist: {distances.flatten()[index]})")
            print("---------------------------------------------------------")
        else:
            print(f"{index}: {book_features_df.index[indices.flatten()[index]]} (dist: {distances.flatten()[index]})")

book_name = input("Enter book name: ")
recommend_item_based(book_name, 10)

In [ ]:
k=int(input("Enter user id:"))
recommend_user_based(k)